In [1]:
import gymnasium as gym
from minigrid.wrappers import RGBImgPartialObsWrapper, ImgObsWrapper
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

/Users/marianaolm/.pyenv/versions/PRIM/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
import my_envs

In [3]:
env_id_train = "MiniGrid-DistributionalShift-Train-v0"
env_id_test  = "MiniGrid-DistributionalShift-Test-v0"

In [4]:
# --- 2. Create and wrap the environment ---
def make_env(env_id, render_mode="rgb_array"):
    env = gym.make(env_id, render_mode=render_mode)
    env = RGBImgPartialObsWrapper(env)
    env = ImgObsWrapper(env)
    return env

In [5]:
# Vectorize for stable-baselines3 compatibility
vec_env = make_vec_env(lambda: make_env(env_id_train), n_envs=4)

In [6]:
# --- 3. Create and train the agent ---
model = PPO("CnnPolicy", vec_env, verbose=1)
model.learn(total_timesteps=10_000)

Using cpu device
Wrapping the env in a VecTransposeImage.


/Users/marianaolm/.pyenv/versions/PRIM/lib/python3.12/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/Users/marianaolm/.pyenv/versions/PRIM/lib/python3.12/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -142     |
| time/              |          |
|    fps             | 2920     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 144         |
|    ep_rew_mean          | -180        |
| time/                   |             |
|    fps                  | 659         |
|    iterations           | 2           |
|    time_elapsed         | 24          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.027879268 |
|    clip_fraction        | 0.327       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.92       |
|    explained_variance   | 0.00124     |
|    learning_rate        | 0.

In [7]:
# --- 4. Test and render one episode ---
test_env = make_env(env_id_test, render_mode="human")
obs, info = test_env.reset()

for step in range(200):
    action, _ = model.predict(obs)
    obs, reward, terminated, truncated, info = test_env.step(action)
    test_env.render() 
    
    if terminated or truncated:
        obs, info = test_env.reset()

test_env.close()
